### Import Libraries

In [1]:
# For data manipulation
import re
import json
import pandas as pd
from pandas import json_normalize

# For NLP
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# For OpenSearch
from opensearchpy import OpenSearch

/Users/ricardo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ricardo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Preparation

In [17]:
from opensearchpy import OpenSearch
from pandas import json_normalize

# Set up OpenSearch connection with environment variables
client = OpenSearch(
    hosts=[{'host': ('62.72.7.91'), 'port': (('9200'))}],
    http_auth=(('admin_jdih'), ('JDIHjuara6065')),
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

# Specify the document ID
document_id = "66-pmk.02-2013"

# Retrieve the document by ID
response = client.get(index="law_analyzer_new4", id=document_id)

# Extract the document source
data = response['_source']

# Normalize the nested data in 'Blocks'
blocks_data = json_normalize(data, record_path=['Blocks'], 
                             meta=['PeraturanId', 'Nomor', 'Slug', 'Judul', 'No', 'Tahun', 'Bentuk', 'Status', 'Bidang', 'Source', 'PeraturanGoId', 'TanggalPenetapan', 'TanggalPengundangan'],
                             record_prefix='Blocks_', errors='ignore')

# Normalize the nested data in 'chunks'
chunks_data = json_normalize(data, record_path=['Blocks', 'chunks'],
                             meta=['PeraturanId', 'Nomor', 'Slug', 'Judul', 'No', 'Tahun', 'Bentuk', 'Status', 'Bidang', 'Source', 'PeraturanGoId', 'TanggalPenetapan', 'TanggalPengundangan', 
                                   ['Blocks', 'content'], ['Blocks', 'context'], ['Blocks', 'type'], ['Blocks', 'ref'], ['Blocks', 'bab'], ['Blocks', 'bagian'], ['Blocks', 'paragraf'], ['Blocks', 'pasal'], ['Blocks', 'source_token_length']],
                             record_prefix='chunks_', errors='ignore')

# Normalize the nested data in 'additional_context'
additional_context_data = json_normalize(data, record_path=['Blocks', 'additional_context'],
                                         meta=['PeraturanId', 'Nomor', 'Slug', 'Judul', 'No', 'Tahun', 'Bentuk', 'Status', 'Bidang', 'Source', 'PeraturanGoId', 'TanggalPenetapan', 'TanggalPengundangan',
                                               ['Blocks', 'content'], ['Blocks', 'context'], ['Blocks', 'type'], ['Blocks', 'ref'], ['Blocks', 'bab'], ['Blocks', 'bagian'], ['Blocks', 'paragraf'], ['Blocks', 'pasal'], ['Blocks', 'source_token_length']],
                                         record_prefix='additional_context_', errors='ignore')

# Normalize the top-level data
top_level_data = json_normalize(data)

# Merge the dataframes
merged_data = top_level_data.merge(blocks_data, how='cross') \
                            .merge(chunks_data, how='outer') \
                            .merge(additional_context_data, how='outer')

# Display the merged dataframe
print(merged_data)


KeyError: "Key 'additional_content' not found. If specifying a record_path, all elements of data should have the path."

In [15]:
#list column names
merged_data.columns

Index(['PeraturanId_x', 'Nomor_x', 'Slug_x', 'Judul_x', 'No_x', 'Tahun_x',
       'Bentuk_x', 'Status_x', 'Bidang_x', 'Source_x', 'PeraturanGoId_x',
       'TanggalPenetapan_x', 'TanggalPengundangan_x', 'Blocks', 'content',
       'context', 'type', 'ref', 'bab', 'bagian', 'paragraf', 'pasal',
       'source_token_length', 'chunks', 'additional_context', 'alias', 'term',
       'PeraturanId_y', 'Nomor_y', 'Slug_y', 'Judul_y', 'No_y', 'Tahun_y',
       'Bentuk_y', 'Status_y', 'Bidang_y', 'Source_y', 'PeraturanGoId_y',
       'TanggalPenetapan_y', 'TanggalPengundangan_y'],
      dtype='object')

In [7]:
flat_data['Blocks']

0    [{'content': 'Dalam Peraturan Menteri ini, yan...
Name: Blocks, dtype: object

In [7]:
# Set up the IndoBERT NER model
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model="cahya/bert-base-indonesian-522M", tokenizer="cahya/bert-base-indonesian-522M")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def extract_prohibitions(text):
    """
    Extracts the list of prohibitions from the given text.

    Args:
        text (str): The text from which to extract the prohibitions.

    Returns:
        list: A list of prohibitions extracted from the text.
    """
    prohibition_pattern = re.compile(r'\bDilarang:\s*([^;]+)', re.IGNORECASE)
    prohibitions = prohibition_pattern.findall(text)
    return prohibitions

def extract_dates(text):
    """
    Extracts dates from the given text.

    Args:
        text (str): The text from which dates need to be extracted.

    Returns:
        list: A list of dates extracted from the text.
    """
    date_pattern = re.compile(r'\b(?:\d{1,2}\s(?:Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)\s\d{4})\b')
    return date_pattern.findall(text)

import re

def extract_money(text):
    """
    Extracts money or monetary keywords from the given text, including multiple currencies.

    Args:
        text (str): The text from which to extract money or monetary keywords.

    Returns:
        list: A list of money or monetary keywords extracted from the text, including Rupiah, USD, and other major currencies.
    """
    patterns = {
        'RP': re.compile(r'\b(?:\(?Rp\s*(?:\d{1,3}(?:[,.]\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)\)?|\(?Rp\s*(?:nol|nol,? nol)\)?\s*\(?Rupiah\)?)\b', re.IGNORECASE),
        'USD': re.compile(r"(USD)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.[0-9]{1,4})"),
        'EUR': re.compile(r"(€|EUR)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.[0-9]{1,4})"),
        'GBP': re.compile(r"(£|GBP)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.[0-9]{1,4})"),
        'JPY': re.compile(r"(¥|JPY)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.?[0-9]{0,4})"),  # Yen typically doesn't use decimal places for common transactions
    }

    matches = []
    for currency, pattern in patterns.items():
        found_matches = pattern.findall(text)
        for match in found_matches:
            matches.append(''.join(match))

    # Convert word numbers to digits for Rupiah
    word_to_number = {
        'satu': '1', 'dua': '2', 'tiga': '3', 'empat': '4', 'lima': '5',
        'enam': '6', 'tujuh': '7', 'delapan': '8', 'sembilan': '9', 'nol': '0',
        'ribu': '000', 'juta': '000000', 'miliar': '000000000', 'triliun': '000000000000'
    }

    numerical_matches = []
    for match in matches:
        for word, value in word_to_number.items():
            if 'Rp' in match:  # Apply word-to-number conversion only for Rupiah
                match = match.replace(word, value)
        numerical_matches.append(match)

    return numerical_matches


def apply_ner(text, ner_pipeline, max_length=512):
    """Apply Named Entity Recognition (NER) using the transformers pipeline on the given text.
    This version supports processing of text longer than 512 tokens by splitting the text into
    manageable parts and then combining the results.

    Args:
        text (str): The input text to perform NER on.
        ner_pipeline (pipeline): The NER pipeline for prediction.
        max_length (int): Maximum length of tokens to process in a single call to the NER pipeline.

    Returns:
        list: A list of predicted named entities in the text.
    """
    if not text.strip():  # Check if text is empty
        return []

    # Initialize variables
    split_texts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    ner_results = []

    # Process each split text
    for split_text in split_texts:
        results = ner_pipeline(split_text)
        ner_results.extend(results)

    return ner_results


flat_data = flat_data.dropna(subset=['content'])

def process_record(record, ner_pipeline):
    """Process a record by extracting relevant information.

    Args:
        record (pd.Series): A pandas Series representing a record with 'content' as one of the keys.
        ner_pipeline: The NER pipeline object used for Named Entity Recognition.

    Returns:
        dict: A dictionary containing the processed record with the following keys:
            - 'content': The original content of the record.
            - 'money': A list of extracted money values.
            - 'dates': A list of extracted dates.
            - 'prohibitions': A list of extracted prohibitions.
            - 'named_entities': A list of named entities recognized by NER.
    """
    content = record['content']

    if not content:
        return {
            'content': content,
            'money': [],
            'dates': [],
            'prohibitions': [],
            'named_entities': []
        }

    result_dict = {
        'content': content,
        'money': extract_money(content),
        'dates': extract_dates(content),
        'prohibitions': extract_prohibitions(content),
    }
    ner_results = apply_ner(content, ner_pipeline)

    return result_dict



In [14]:
# Ensure you have a DataFrame 'flat_data' with a 'content' column available.
# Create a new DataFrame to store the processed records
processed_records = [process_record(row, ner_pipeline) for index, row in flat_data.iterrows()]
processed_df = pd.DataFrame(processed_records)

# Now `processed_df` contains the original content and the extracted information.
print(processed_df)  # To inspect the first few records.

                                                content money          dates  \
0                                                          []             []   
1     BERITA NEGARA REPUBLIK INDONESIA No.920, 2017 ...    []             []   
2     PERATURAN MENTERI KEUANGAN REPUBLIK INDONESIA ...    []             []   
3                     DENGAN RAHMAT TUHAN YANG MAHA ESA    []             []   
4                  MENTERI KEUANGAN REPUBLIK INDONESIA,    []             []   
...                                                 ...   ...            ...   
2342                              SRI MULYANI INDRAWATI    []             []   
2343    Diundangkan di Jakarta pada tanggal 7 Juli 2017    []  [7 Juli 2017]   
2344  DIREKTUR JENDERAL PERATURAN PERUNDANG-UNDANGAN...    []             []   
2345                                                ttd    []             []   
2346                                 WIDODO EKATJAHJANA    []             []   

     prohibitions named_entities  
0   

In [16]:
# #drop named_entities column
# processed_df = processed_df.drop(columns='named_entities')
for col in processed_df.columns:
    processed_df[col] = processed_df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [18]:
#unique column money, dates, prohibitions and print all of them with for loop
for col in ['money', 'dates', 'prohibitions']:
    print(f'Unique values in {col} column:')
    print(processed_df[col].unique())
    print()

Unique values in money column:
['' 'Rp0, Rp0']

Unique values in dates column:
['' '8 Juni 2017' '5 Juli 2017' '7 Juli 2017' '16 Januari 2023'
 '17 Januari 2023' '23 April 2019' '18 Oktober 2019']

Unique values in prohibitions column:
[''
 'a. menggunakan kewenangan yang dimiliki untuk kepentingan pribadi, keluarga, dan/atau golongan']



In [19]:
processed_df.to_csv('processed_data.csv', index=False)

In [ ]:
# For data manipulation
import re
import json
import pandas as pd
from pandas import json_normalize

# For NLP
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# For OpenSearch
from opensearchpy import OpenSearch

# Set up OpenSearch connection with environment variables
client = OpenSearch(
    hosts=[{'host': ('62.72.7.91'), 'port': (('9200'))}],
    http_auth=(('admin_jdih'), ('JDIHjuara6065')),
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)

client.info()

# Specify the document ID
document_id = "66-pmk.02-2013"

# Retrieve the document by ID
response = client.get(index="law_analyzer_new4", id=document_id)

# Extract the document source
data = response['_source']

flat_data = json_normalize(data)  # flatten the data

# Separate the 'Blocks' column into new columns
blocks_data = json_normalize(flat_data['Blocks'].explode())
flat_data = pd.concat([flat_data, blocks_data], axis=1)
flat_data = flat_data.drop(columns=['Blocks'])

flat_data.info()

# Set up the IndoBERT NER model
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model="cahya/bert-base-indonesian-522M", tokenizer="cahya/bert-base-indonesian-522M")

def extract_prohibitions(text):
    """
    Extracts the list of prohibitions from the given text.

    Args:
        text (str): The text from which to extract the prohibitions.

    Returns:
        list: A list of prohibitions extracted from the text.
    """
    prohibition_pattern = re.compile(r'(?:dilarang|Dilarang):\s*([a-z](?:(?!(?:dilarang|Dilarang):).)*)\.', re.IGNORECASE | re.DOTALL)
    prohibitions = prohibition_pattern.findall(text)

    cleaned_prohibitions = []
    for prohibition in prohibitions:
        cleaned_prohibition = re.sub(r'\n', ' ', prohibition).strip()
        cleaned_prohibitions.append(cleaned_prohibition)

    return cleaned_prohibitions

def extract_dates(text):
    """
    Extracts dates from the given text.

    Args:
        text (str): The text from which dates need to be extracted.

    Returns:
        list: A list of dates extracted from the text.
    """
    date_pattern = re.compile(r'\b(?:\d{1,2}\s(?:Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)\s\d{4})\b')
    return date_pattern.findall(text)

def extract_money(text):
    """
    Extracts money or monetary keywords from the given text, including multiple currencies.

    Args:
        text (str): The text from which to extract money or monetary keywords.

    Returns:
        list: A list of money or monetary keywords extracted from the text, including Rupiah, USD, and other major currencies.
    """
    patterns = {
        'RP': re.compile(r'\b(?:\(?Rp\s*(?:\d{1,3}(?:[,.]\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)\)?|\(?Rp\s*(?:nol|nol,? nol)\)?\s*\(?Rupiah\)?)\b', re.IGNORECASE),
        'USD': re.compile(r"(USD)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.[0-9]{1,4})"),
        'EUR': re.compile(r"(€|EUR)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.[0-9]{1,4})"),
        'GBP': re.compile(r"(£|GBP)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.[0-9]{1,4})"),
        'JPY': re.compile(r"(¥|JPY)([+-]?[0-9]{1,3}(,?[0-9]{3})*)(\.?[0-9]{0,4})"),  # Yen typically doesn't use decimal places for common transactions
    }

    matches = []
    for currency, pattern in patterns.items():
        found_matches = pattern.findall(text)
        for match in found_matches:
            matches.append(''.join(match))

    # Convert word numbers to digits for Rupiah
    word_to_number = {
        'satu': '1', 'dua': '2', 'tiga': '3', 'empat': '4', 'lima': '5',
        'enam': '6', 'tujuh': '7', 'delapan': '8', 'sembilan': '9', 'nol': '0',
        'ribu': '000', 'juta': '000000', 'miliar': '000000000', 'triliun': '000000000000'
    }

    numerical_matches = []
    for match in matches:
        for word, value in word_to_number.items():
            if 'Rp' in match:  # Apply word-to-number conversion only for Rupiah
                match = match.replace(word, value)
        numerical_matches.append(match)

    return numerical_matches


def apply_ner(text, ner_pipeline, max_length=512):
    """Apply Named Entity Recognition (NER) using the transformers pipeline on the given text.
    This version supports processing of text longer than 512 tokens by splitting the text into
    manageable parts and then combining the results.

    Args:
        text (str): The input text to perform NER on.
        ner_pipeline (pipeline): The NER pipeline for prediction.
        max_length (int): Maximum length of tokens to process in a single call to the NER pipeline.

    Returns:
        list: A list of predicted named entities in the text.
    """
    if not text.strip():  # Check if text is empty
        return []

    # Initialize variables
    split_texts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    ner_results = []

    # Process each split text
    for split_text in split_texts:
        results = ner_pipeline(split_text)
        ner_results.extend(results)

    return ner_results


flat_data = flat_data.dropna(subset=['content'])

def process_record(record, ner_pipeline):
    """Process a record by extracting relevant information.

    Args:
        record (pd.Series): A pandas Series representing a record with 'content' as one of the keys.
        ner_pipeline: The NER pipeline object used for Named Entity Recognition.

    Returns:
        dict: A dictionary containing the processed record with the following keys:
            - 'content': The original content of the record.
            - 'money': A list of extracted money values.
            - 'dates': A list of extracted dates.
            - 'prohibitions': A list of extracted prohibitions.
            - 'named_entities': A list of named entities recognized by NER.
    """
    content = record['content']

    if not content:
        return {
            'content': content,
            'money': [],
            'dates': [],
            'prohibitions': [],
            'named_entities': []
        }

    result_dict = {
        'content': content,
        'money': extract_money(content),
        'dates': extract_dates(content),
        'prohibitions': extract_prohibitions(content),
    }
    ner_results = apply_ner(content, ner_pipeline)

    return result_dict


# Ensure you have a DataFrame 'flat_data' with a 'content' column available.
# Create a new DataFrame to store the processed records
processed_records = [process_record(row, ner_pipeline) for index, row in flat_data.iterrows()]
processed_df = pd.DataFrame(processed_records)

# Now `processed_df` contains the original content and the extracted information.
print(processed_df)  # To inspect the first few records.

for col in processed_df.columns:
    processed_df[col] = processed_df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Unique values in money, dates, and prohibitions columns
for col in ['money', 'dates', 'prohibitions']:
    print(f'Unique values in {col} column:')
    print(processed_df[col].unique())
    print()

processed_df.to_csv('processed_data.csv', index=False)
